# NIRI Imaging - Data Reduction Tutorial

Found a good, complete dataset.  GN-2015B-Q-31-118.  Small dither and offset to sky because there's a galaxy in the bottom right corner.  It has the darks for the science, no darks for the standard.  It has the standard.  Lamp-on, lamp-off flats.   I'll have to dig for short darks.   H-band.   Certainly worth a try.  It's pretty too.   It's a supernova observation.

* Science:  N20160102S0270-279  (270-274 on target, 275-279 on sky)
* Science Darks: N20160102S0423-432   (20 sec, like the Science)

* Flats, lamp-on: N20160102S0373-382
* Flats, lamp-off: N20160102S0363-372

* Standard: N20160102S0295-299

* Short darks: N20160103S0463-472   (To find them, set a date range around date of science observations, Instrument NIRI, Obs.Type set to DARK, and exposure time set 1 second.)

## Create the various file lists

DRAGONS has a convenience tool, `typewalk`, that can identify data based on their tags.  It is quite useful when it comes to creating file lists that will serve as inputs to `reduce`.

It is not quite the perfect tool though.  We will be creating a tool better suited to creating `reduce` input lists but it is not available yet.  So, this step will require an editor and some fiddling.  (We're not happy about it either.)  


<hr>

** Darks **: We have darks for the science target, with an exposure time equal to that of the science target observations.  We also have 1-second darks that are used for creating the bad pixel mask (BPM).   Darks are not obtained for the standard star observations.

** Flats **: There are lamp-on and lamp-off flats.  They are used to create a master flat field that will be applied to standard star and the science target.  They are also used to create the BPM.

** Objects **: We have standard star observations and science target observations.

In [ ]:
%%bash

# cd to the playground directory
cd /data/workspace/niri_tutorial/playground
pwd


In [ ]:
%%bash
cd /data/workspace/niri_tutorial/playground


# Create the dark frames lists. 
typewalk --tags=DARK -d ../playdata -o alldarks.lis

# 'typewalk' cannot differentiate based on the exposure time.
# Action: Edit manually to produce darks1s.lis and darks10s.lis
cp alldarks.lis darks1s.lis
cp alldarks.lis darks20s.lis  
# vi darks1s.lis    # N20160103S0463-472
# vi darks20s.lis   # N20160102S0423-432

In [ ]:
%%bash
cd /data/workspace/niri_tutorial/playground

# Create the flat field list
typewalk --tags=FLAT -d ../playdata -o allflats.lis

# Create the standard and target list
typewalk --tags=IMAGE --xtags=FLAT -d ../playdata -o allobjects.lis

# 'typewalk' cannot differentiate based on object name.
# Action: Edit manually to produce stdstar.lis and target.lis
cp allobjects.lis stdstar.lis
cp allobjects.lis target.lis  
# vi stdstar.lis    # N20160102S0295-299
# vi target.lis     # N20160102S0270-279


## Science reduction using the Calibration Manager

### Setup the calibration manager

    [calibs]
    standalone = True
    database_dir = /data/workspace/niri_tutorial/playground

In [ ]:
%%bash

# Configure the calibration manager.
# vi ~/.geminidr/rsys.cfg

# Initialize the calibration database
caldb init


### Reduce data

In [ ]:
%%bash
cd /data/workspace/niri_tutorial/playground

# -----------------------------------------------
# Create the master dark for the science target
reduce @darks20s.lis

# Add the dark to the calibration manager.
caldb add N20160102S0423_dark.fits


# -----------------------------------------------
# Create the user Bad Pixel Mask (BPM)
reduce @allflats.lis @darks1s.lis -r makeProcessedBPM


# -----------------------------------------------
# Create the master flat field
reduce @allflats.lis -p addDQ:user_bpm=N20160102S0373_bpm.fits

# Add the flat to the calibration manager.
caldb add N20160102S0373_flat.fits


# -----------------------------------------------
# Reduce the standard star
reduce @stdstar.lis -p addDQ:user_bpm=N20160102S0373_bpm.fits darkCorrect:do_dark=False


# -----------------------------------------------
# Reduce the science target
#   We need to set the scaling of the sky to False because the target 
#   fills the field of view and does not represent a reasonable background.
#   Scaling results in an oversubtraction of the sky frame.
reduce @target.lis -p addDQ:user_bpm=N20160102S0373_bpm.fits skyCorrect:scale=False


## Science reduction setting calibrations manually

In [ ]:
%%bash

#Create the master dark for the science target
#Create the user Bad Pixel Mask (BPM)
#Create the master flat field
#Reduce the standard star
reduce @stdstar.lis -p addDQ:user_bpm=N20180106S0681_bpm.fits --user_cal processed_flat:N20180106S0681_flat.fits


#Reduce the science target